In [360]:
import os
import scipy.io as sio
import numpy as np
from itertools import product
from collections import namedtuple
from joblib import Parallel, delayed
import emg_features

ModuleNotFoundError: No module named 'nitime'

In [301]:
# subjects = list(range(0,27))  #27 person
subjects=[0] # for test only person 0
# gestures = list(range(1,53))  #52 different gestures
gestures=[1] #for test only gesture 1
trials = list(range(10))      #10 channel

In [302]:
source_path='C:/Users/11488/Desktop/my semg/data/db-1/s1'

### Parameter

In [348]:
filtering_type = 'lowpass'
framerate = 100

window_length_ms = 200
window_stride_ms = 10

window = int(window_length_ms*framerate/1000)   #20
stride = int(window_stride_ms*framerate/1000)   #1


In [304]:
output_path = ('C:/Users/11488/Desktop/my semg/wwt/ninapro-feature/ninapro-db1-var-raw-prepro-%s-win-%d-stride-%d' % (filtering_type, window, stride))

In [305]:
Combo = namedtuple('Combo', ['subject', 'gesture', 'trial']) #define a class Comb with attribute "subject" "gesture" "trial"

In [306]:
feature_list = ['ssc', 'var']  #slop sign change, variance

### Process My Data

In [307]:
#load my data
print('load S1_A1_E1.mat')
data1 = sio.loadmat('C:/Users/11488/Desktop/my semg/data/db-1/s1/S1_A1_E1.mat')
# print(data1.keys())
# print(data1['emg'].shape)
# print(data1['stimulus'].shape)
# print(data1['stimulus'])
# data1['emg']
print('load S1_A1_E2.mat')
data2 = sio.loadmat('C:/Users/11488/Desktop/my semg/data/db-1/s1/S1_A1_E2.mat')

print('load S1_A1_E3.mat')
data3 = sio.loadmat('C:/Users/11488/Desktop/my semg/data/db-1/s1/S1_A1_E3.mat')

load S1_A1_E1.mat
load S1_A1_E2.mat
load S1_A1_E3.mat


In [308]:
combos = get_combos(product(subjects, gestures, trials))        
combos = list(combos)
# print(combos)
for combo in combos:
#     print(combo.subject)
    out_dir=os.path.join(source_path,
                      '{c.subject:03d}',
                      '{c.gesture:03d}').format(c=combo)
    if os.path.isdir(out_dir) is False:
             os.makedirs(out_dir)
    path=os.path.join(out_dir,
                      '{c.subject:03d}_{c.gesture:03d}_{c.trial:03d}.mat').format(c=combo)
    ch=dict()
    d=[]
    for i in range( data1['stimulus'].shape[0]):
        if data1['stimulus'][i]==[1]:
            d.append(data1['emg'][i][combo.trial])
    for i in range( data2['stimulus'].shape[0]):
        if data2['stimulus'][i]==[1]:
            d.append(data2['emg'][i][combo.trial])
    for i in range( data3['stimulus'].shape[0]):
        if data3['stimulus'][i]==[1]:
            d.append(data3['emg'][i][combo.trial])
    ch['data']=d
    sio.savemat(path, ch)   
            
        
    

In [309]:
# path

In [310]:
# sio.savemat('C:/Users/11488/Desktop/my semg/data/db-1/s1/000_001_000.mat', dict())  

### Loand and Check

In [311]:
combos = get_combos(product(subjects, gestures, trials))        
combos = list(combos)
# source_path='C:/Users/11488/Desktop/my semg/data/db-1/s1'
# print(combos)
data_set=[[]for i in range(len(trials))]
for combo in combos:
    in_path = os.path.join(
                    source_path,
                    '{c.subject:03d}',
                    '{c.gesture:03d}',
                    '{c.subject:03d}_{c.gesture:03d}_{c.trial:03d}.mat').format(c=combo)
#     print(in_path)
    tr=sio.loadmat(in_path)
    data_set[combo.trial]=tr['data']
    print('tr'+str(combo.trial)+'shape:',tr['data'].shape )

tr0shape: (1, 15476)
tr1shape: (1, 15476)
tr2shape: (1, 15476)
tr3shape: (1, 15476)
tr4shape: (1, 15476)
tr5shape: (1, 15476)
tr6shape: (1, 15476)
tr7shape: (1, 15476)
tr8shape: (1, 15476)
tr9shape: (1, 15476)


In [314]:
data_set

[array([[0.0146, 0.0171, 0.0342, ..., 0.0024, 0.0049, 0.0024]]),
 array([[0.0024, 0.0024, 0.0024, ..., 0.0024, 0.0024, 0.0024]]),
 array([[0.0049, 0.0024, 0.0024, ..., 0.0024, 0.0024, 0.0024]]),
 array([[0.0024, 0.0024, 0.0024, ..., 0.0024, 0.0024, 0.0024]]),
 array([[0.0024, 0.0024, 0.0024, ..., 0.0024, 0.0024, 0.0024]]),
 array([[0.0024, 0.0024, 0.0024, ..., 0.0024, 0.0024, 0.0024]]),
 array([[0.0024, 0.0024, 0.0024, ..., 0.0073, 0.0049, 0.0024]]),
 array([[0.0171, 0.022 , 0.0269, ..., 0.1196, 0.1172, 0.1123]]),
 array([[0.0024, 0.0024, 0.0024, ..., 0.0024, 0.0024, 0.0024]]),
 array([[0.0024, 0.0024, 0.0024, ..., 0.0488, 0.0488, 0.0464]])]

### Define Function

In [315]:
def get_combos(*args):
    for arg in args:
        if isinstance(arg, tuple):
            arg = [arg]
        for a in arg:
            yield Combo(*a)
           

In [316]:
def butter_lowpass_filter(data, cut, fs, order, zero_phase=False):
    from scipy.signal import butter, lfilter, filtfilt

    nyq = 0.5 * fs
    cut = cut / nyq

    b, a = butter(order, cut, btype='low')
    y = (filtfilt if zero_phase else lfilter)(b, a, data)
    return y

In [317]:
def get_segments(data, window, stride):
    return windowed_view(
        data.flat,
        window * data.shape[1],
        (window-stride)* data.shape[1]
    )

def windowed_view(arr, window, overlap):
    from numpy.lib.stride_tricks import as_strided
    arr = np.asarray(arr)
    window_step = window - overlap
    new_shape = arr.shape[:-1] + ((arr.shape[-1] - overlap) // window_step,
                                  window)
    new_strides = (arr.strides[:-1] + (window_step * arr.strides[-1],) +
                   arr.strides[-1:])
    return as_strided(arr, shape=new_shape, strides=new_strides)

In [274]:
# for i in combos:
#     in_path = os.path.join(
#                     input_path, 'data',
#                     '{c.subject:03d}',
#                     '{c.gesture:03d}',
#                     '{c.subject:03d}_{c.gesture:03d}_{c.trial:03d}.mat').format(c=i)
#     out_dir = os.path.join(
#                     output_path,
#                     '{c.subject:03d}',
#                     '{c.gesture:03d}').format(c=i)  
#     print(out_dir)

In [358]:
combo=combos[0]
in_path = os.path.join(
                source_path,
                '{c.subject:03d}',
                '{c.gesture:03d}',
                '{c.subject:03d}_{c.gesture:03d}_{c.trial:03d}.mat').format(c=combo)
out_dir = os.path.join(
                    output_path,
                    '{c.subject:03d}',
                    '{c.gesture:03d}').format(c=combo)  
                
if os.path.isdir(out_dir) is False:
    os.makedirs(out_dir)               
data = sio.loadmat(in_path)['data'].astype(np.float32)
print ("Subject %d Gesture %d Trial %d data loaded!" % (combo.subject, combo.gesture, combo.trial))

if filtering_type == 'lowpass':            
#             data = np.transpose([lowpass(ch, 1, framerate, 1, zero_phase=True) for ch in data.T])
    data = np.transpose([butter_lowpass_filter(ch, 1, framerate, 1, zero_phase=True) for ch in data])   
    print ("Subject %d Gesture %d Trial %d bandpass filtering finished!" % (combo.subject, combo.gesture, combo.trial))
else:
    pass
chnum = data.shape[1] 

data = get_segments(data, window, stride)
data = data.reshape(-1, window, chnum)


Subject 0 Gesture 1 Trial 0 data loaded!
Subject 0 Gesture 1 Trial 0 bandpass filtering finished!


(15457, 20, 1)

In [253]:
in_path

'C:/Users/11488/Desktop/my semg/data\\data\\000\\001\\000_001_000.mat'

In [ ]:
def emg_feature_extraction_parallel_2(input_path, output_path, combo, feature_list):
    in_path = os.path.join(
                input_path, 'data',
                '{c.subject:03d}',
                '{c.gesture:03d}',
                '{c.subject:03d}_{c.gesture:03d}_{c.trial:03d}.mat').format(c=combo)
                
    out_dir = os.path.join(
                    output_path,
                    '{c.subject:03d}',
                    '{c.gesture:03d}').format(c=combo)  
                
    if os.path.isdir(out_dir) is False:
             os.makedirs(out_dir)                 
     
                
    data = sio.loadmat(in_path)['data'].astype(np.float32)
        
    print ("Subject %d Gesture %d Trial %d data loaded!" % (combo.subject, combo.gesture, combo.trial))
      
        
      
    if filtering_type is 'lowpass':            
#             data = np.transpose([lowpass(ch, 1, framerate, 1, zero_phase=True) for ch in data.T])
        data = np.transpose([butter_lowpass_filter(ch, 1, framerate, 1, zero_phase=True) for ch in data.T])   
        print ("Subject %d Gesture %d Trial %d bandpass filtering finished!" % (combo.subject, combo.gesture, combo.trial))
    else:
        pass
           
#      data = downsample(data, step=20)        
        
    chnum = data.shape[1]    
    data = get_segments(data, window, stride)
      data = data.reshape(-1, window, chnum)
      
      for feature_name in feature_list:
          feature = [np.transpose(extract_emg_feature(seg.T, feature_name)) for seg in data]
          feature = np.array(feature)
          out_path = os.path.join(
                                  out_dir,
                                  '{0.subject:03d}_{0.gesture:03d}_{0.trial:03d}_{1}.mat').format(combo, feature_name)  
          sio.savemat(out_path, {'data': feature, 'label': combo.gesture, 'subject': combo.subject, 'trial':combo.trial})                        
          print ("Subject %d Gesture %d Trial %d %s saved!" % (combo.subject, combo.gesture, combo.trial, feature_name))


In [244]:
print ("NinaPro feature map generation, use window = %d frames, stride = %d frames" % (window, stride)) 

combos = get_combos(product(subjects, gestures, trials))
           
combos = list(combos)

# combos

NinaPro feature map generation, use window = 20 frames, stride = 1 frames


In [ ]:
Parallel(n_jobs=8)(delayed(emg_feature_extraction_parallel_2)(input_path, output_path, combo, feature_list) for combo in combos) 